In [1]:
import sys, os
import numpy as np
import pandas as pd
import pygama

%load_ext line_profiler

sys.path.append("/Users/wisecg/dev/mj60")
from test_pygama_2 import make_processor_list, tier0

run = 42343
raw_file = "/Users/wisecg/dev/mj60/data/2018-10-9-P3LTP_Run42343"
t1_file = "/Users/wisecg/dev/mj60/data/t1_run42343.h5"

digitizer = pygama.decoders.digitizers.Gretina4MDecoder(
    correct_presum = False,
    split_waveform = False,
)

In [11]:
# %%capture
n_evt = 10000
tier0(run, raw_file, n_evt)
event_df = pd.read_hdf(t1_file, key=digitizer.decoder_name)

Starting pygama Tier 0 processing ...
  Input file: /Users/wisecg/dev/mj60/data/2018-10-9-P3LTP_Run42343
Header parsed.
   48310 longs (in plist header)
   193230 bytes in the header
Total file size: 2001.221 MB
Run number: 42343
The Data IDs present in this file (header) are:
    1: OR1DHistoDecoder
    14: ORGretinaTriggerDecoder
    2: ORMJDPreAmpDecoderForAdc
    4: ORScriptDecoderForRecord
    3: ORScriptDecoderForState
    9: ORCV830DecoderForEvent
    10: ORCV830DecoderForPolledRead
    11: ORCAEN792DecoderForQdc
    12: ORCAEN792NDecoderForQdc
    15: ORiSegHVCardDecoderForHV
    5: ORiSegHVCardDecoderForHV
    6: ORGretina4MWaveformDecoder
    13: ORHPPulserDecoderForPulserSettings
    16: ORRunDecoderForRun
Available pygama decoders:
    -- ORGretina4MWaveformDecoder
    -- ORSIS3302DecoderForEnergy
    -- ORMJDPreAmpDecoderForAdc
    -- ORiSegHVCardDecoderForHV
Applying these decoders to the file:
    6: ORGretina4MWaveformDecoder
    5: ORiSegHVCardDecoderForHV
    2: ORMJD

In [6]:
# %timeit 
event_df = pd.read_hdf(t1_file, key=digitizer.decoder_name)

In [7]:
# declare the Tier 1 processor list
proc_list = make_processor_list()
proc_list.digitizer = digitizer
proc_list.max_event_number = len(event_df)
proc_list.verbose = True
proc_list.t0_cols = event_df.columns.tolist()
proc_list.runNumber = run
proc_list.num = 0

In [ ]:
# save a new dataframe where each row is its own wf
wfs = event_df["waveform"]
wf_df = pd.DataFrame.from_dict(dict(zip(wfs.index, wfs.values))).T
# wf_df.to_hdf("t2_wfs.h5", key="data", format='table', mode='w')
# wf_df = pd.read_hdf("t2_wfs.h5")
print(event_df["waveform"].shape, wf_df.shape)

In [8]:
import time
t_start = time.clock()
t1_df = event_df.apply(proc_list.Process, axis=1)
print("Elapsed:",time.clock()-t_start,"seconds")

PROGRESS : [####################] 99.982%  (Run 42343)Elapsed: 68.58286100000001 seconds


In [ ]:
%timeit t1_df = event_df.apply(proc_list.Process, axis=1)

In [ ]:
%lprun?

In [ ]:
# show line profile of 'apply'
%lprun -f event_df.apply event_df.apply(proc_list.Process, axis=1)

In [ ]:
# show line profile of the proc list
%lprun -f proc_list.Process event_df.apply(proc_list.Process, axis=1)

In [ ]:
%timeit t1_df = wf_df.apply(proc_list.Process, axis=1, flat=True)

In [ ]:
def fit_baseline(waveform, start_index=0, end_index=500, order=1):
    """ Finds baseline from start index to end index samples (default linear) """
    if end_index == -1: end_index = len(waveform)
    p = np.polyfit(
        np.arange(start_index, end_index), waveform[start_index:end_index], 1)
    return p

In [ ]:
%%timeit
new_rows = []
for i, wf in event_df.iterrows():
    mywf = wf["waveform"] # this is an ndarray, (2032, )
    new_rows.append(fit_baseline(mywf))
t1_df = pd.DataFrame(new_rows)

In [ ]:
%%timeit
new_rows = []
for i, wf in wf_df.iterrows():
    new_rows.append(fit_baseline(wf.values)) # put the ndarray in here
t1_df = pd.DataFrame(new_rows)

In [ ]:
wf_df.values

In [ ]:
def fit_baseline_vec(arr):

    # super simple, mean baseline, with correct shape (should be vectorized)
    # return np.mean(arr[:,:500], axis=1) 
        
    # This block looks good!  just have to check it.
    x = np.arange(0, 500)
    y = arr[:, 0:500].T
    p = np.polynomial.polynomial.polyfit(x, y, 1).T
    
    # print("vectorized result")
    # print(arr.shape)
    # print(p.shape)
    # print(p)

    # # looks good except the 1st and 2nd values are switched
    # print("loop over wfs")
    # for wf in y.T:
    #     p = np.polyfit(x, wf, 1)
    #     print(p)
    
    return p

result = fit_baseline_vec(wf_df.values[0:10])
# print(result.shape)
# print(result)

In [ ]:
vals = np.array([[-1.56113514,  4.51759732],
       [-2.80261623,  5.068371  ],
       [ 0.7792729 ,  6.0169462 ],
       [-1.35672858,  3.52517478],
       [-1.92074891,  5.79966161],
       [-2.79340321,  4.73430001],
       [-2.79655868,  5.05361163],
       [-2.13637747,  5.39255837],
       [ 0.17341809,  3.60918261],
       [-1.22712921,  4.95327158]])

def function(array):
    c1 = array[:, 0]
    c2 = array[:, 1]
    return c1 * c2

print(function(vals), function(vals).shape)

In [ ]:
wfs = wf_df.values
t1 = fit_baseline_vec(wfs)
t1_df = pd.DataFrame(t1)

t1_df = wf_df.apply(fit_baseline, axis=1)

In [ ]:
%timeit t1_df = pd.DataFrame(fit_baseline_vec(wf_df.values))

In [ ]:
%timeit t1_df = wf_df.apply(fit_baseline, axis=1)

In [ ]:
%lprun -f fit_baseline_vec pd.DataFrame(fit_baseline_vec(wf_df.values))

In [9]:
# ++++++++++++++++++++++++++++++++++++++++

# try vectorized processor

digitizer = pygama.decoders.digitizers.Gretina4MDecoder(
        correct_presum = False,
        split_waveform = False,
        )
event_df = pd.read_hdf(t1_file, key=digitizer.decoder_name)

vec_procs = pygama.VectorizedProcessorList()

vec_procs.AddCalculator(pygama.processing.vectorized.fit_baseline,
                            in_name = ["waveform"],
                            out_name = ["bl_int", "bl_slope"],
                            fun_args = {"end_index":700})

# vec_procs.AddCalculator(pygama.processing.vectorized.avg_baseline,
#                         in_name = ["waveform"],
#                         out_name = ["bl_avg"],
#                         fun_args = {"end_index":700})

In [ ]:
t_start = time.clock()
t1_df = vec_procs.Process(event_df)
print("Vec elapsed:",time.clock()-t_start)

In [10]:
%timeit t1_df = vec_procs.Process(event_df)

Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
Applying processor: <function fit_baseline at 0x1105d6158>
1.6 s ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(t1_df)

In [3]:
%lprun -f vec_procs.Process vec_procs.Process(event_df)

Applying processor: <function fit_baseline at 0x1105d6158>
